In [1]:
# Import necessary libraries
import os
from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm # For OpenAI support
from google.adk.tools import ToolContext
import json

# Convenience libraries for working with Neo4j inside of Google ADK
from neo4j_for_adk import graphdb, tool_success, tool_error

import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.CRITICAL)

print("Libraries imported.")

Libraries imported.


In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

# Defining Model Constants for easier use 
MODEL_GPT = "openai/gpt-4o"

llm = LiteLlm(model=MODEL_GPT)

# Test LLM with a direct call
response = llm.llm_client.completion(
    model=llm.model,
    api_key=os.getenv("OPENAI_API_KEY"),
    messages=[{"role": "user", "content": "Are you ready?"}],
    tools=[]
)
print(json.dumps(response.dict(), indent=2))

print("\nOpenAI is ready for use.")

{
  "id": "chatcmpl-CA9V0SqlwvZVjOcSgiTgHVdJ137YE",
  "created": 1756535830,
  "model": "gpt-4o-2024-08-06",
  "object": "chat.completion",
  "system_fingerprint": "fp_80956533cb",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Yes, I'm ready! How can I assist you today?",
        "role": "assistant",
        "tool_calls": null,
        "function_call": null,
        "annotations": []
      },
      "provider_specific_fields": {}
    }
  ],
  "usage": {
    "completion_tokens": 13,
    "prompt_tokens": 27,
    "total_tokens": 40,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0,
      "text_tokens": null
    },
    "prompt_tokens_details": {
      "audio_tokens": 0,
      "cached_tokens": 0,
      "text_tokens": null,
      "image_tokens": null
    }
  },
  "service_tier": "default"
}

OpenAI is ready for use.

## User Intent Agent

###  Definition

In [3]:
user_intent_agent_role_and_goal = """
    You are an expert at knowledge graph use cases. 
    Your primary goal is to help the user come up with a knowledge graph use case.
"""

user_intent_agent_conversational_hints = """
    If the user is unsure what to do, make some suggestions based on classic use cases like:
    - social network involving friends, family, or professional relationships
    - logistics network with suppliers, customers, and partners
    - recommendation system with customers, products, and purchase patterns
    - fraud detection over multiple accounts with suspicious patterns of transactions
    - pop-culture graphs with movies, books, or music
"""

user_intent_agent_output_definition = """
    A user goal has two components:
    - kind_of_graph: at most 3 words describing the graph, for example "social network" or "USA freight logistics"
    - description: a few sentences about the intention of the graph, for example "A dynamic routing and delivery system for cargo." or "Analysis of product dependencies and supplier alternatives."
"""

user_intent_agent_chain_of_thought_directions = """
    Think carefully and collaborate with the user:
    1. Understand the user's goal, which is a kind_of_graph with description
    2. Ask clarifying questions as needed
    3. When you think you understand their goal, use the 'set_perceived_user_goal' tool to record your perception
    4. Present the perceived user goal to the user for confirmation
    5. If the user agrees, use the 'approve_perceived_user_goal' tool to approve the user goal. This will save the goal in state under the 'approved_user_goal' key.
"""

In [4]:
user_intent_agent_instruction = f"""
{user_intent_agent_role_and_goal}
{user_intent_agent_conversational_hints}
{user_intent_agent_output_definition}
{user_intent_agent_chain_of_thought_directions}
"""

print(f"User Intent Agent instructions defined. {user_intent_agent_instruction}")

User Intent Agent instructions defined. 

    You are an expert at knowledge graph use cases. 
    Your primary goal is to help the user come up with a knowledge graph use case.


    If the user is unsure what to do, make some suggestions based on classic use cases like:
    - social network involving friends, family, or professional relationships
    - logistics network with suppliers, customers, and partners
    - recommendation system with customers, products, and purchase patterns
    - fraud detection over multiple accounts with suspicious patterns of transactions
    - pop-culture graphs with movies, books, or music


    A user goal has two components:
    - kind_of_graph: at most 3 words describing the graph, for example "social network" or "USA freight logistics"
    - description: a few sentences about the intention of the graph, for example "A dynamic routing and delivery system for cargo." or "Analysis of product dependencies and supplier alternatives."


    Think careful

### Tool Definition

In [5]:
# Tool: Set Perceived User Goal
# to encourage collaboration with the user, the first tool only sets the perceived user goal

PERCEIVED_USER_GOAL = "perceived_user_goal"

def set_perceived_user_goal(kind_of_graph: str, graph_description:str, tool_context: ToolContext):
    """Sets the perceived user's goal, including the kind of graph and its description.
    
    Args:
        kind_of_graph: 2-3 word definition of the kind of graph, for example "recent US patents"
        graph_description: a single paragraph description of the graph, summarizing the user's intent
    """
    user_goal_data = {"kind_of_graph": kind_of_graph, "graph_description": graph_description}
    tool_context.state[PERCEIVED_USER_GOAL] = user_goal_data
    return tool_success(PERCEIVED_USER_GOAL, user_goal_data)

In [6]:
# Tool: Approve the perceived user goal
# approval from the user should trigger a call to this tool

APPROVED_USER_GOAL = "approved_user_goal"

def approve_perceived_user_goal(dummy: str = "", tool_context: ToolContext = None):
    """Upon approval from user, will record the perceived user goal as the approved user goal.

    Only call this tool if the user has explicitly approved the perceived user goal.
    """
    if tool_context is None:
        return tool_error("Tool context is required.")

    if PERCEIVED_USER_GOAL not in tool_context.state:
        return tool_error("perceived_user_goal not set. Set perceived user goal first, or ask clarifying questions if you are unsure.")

    tool_context.state[APPROVED_USER_GOAL] = tool_context.state[PERCEIVED_USER_GOAL]
    
    return tool_success(APPROVED_USER_GOAL, tool_context.state[APPROVED_USER_GOAL])

In [ ]:
# Tool: Get the approved user goal
def get_approved_user_goal(tool_context: ToolContext):
    """Returns the approved user goal from the tool context state, if it exists."""
    APPROVED_USER_GOAL = "approved_user_goal"
    if APPROVED_USER_GOAL not in tool_context.state:
        return tool_error("No approved user goal found. Approve a user goal first.")
    return tool_success(APPROVED_USER_GOAL, tool_context.state[APPROVED_USER_GOAL])


In [7]:
user_intent_agent_tools = [set_perceived_user_goal, approve_perceived_user_goal]

### Agent Definition

In [8]:
# Finally, construct the agent

user_intent_agent = Agent(
    name="user_intent_agent_v1", # a unique, versioned name
    model=llm, # defined earlier in a variable
    description="Helps the user ideate on a knowledge graph use case.", # used for delegation
    instruction=user_intent_agent_instruction, # the complete instructions you composed earlier
    tools=user_intent_agent_tools, # the list of tools
)

print(f"Agent '{user_intent_agent.name}' created.")

Agent 'user_intent_agent_v1' created.


In [9]:
# use a helper to create an agent execution environment
from helper import make_agent_caller

# NOTE: if re-running the session, come back here to re-initialize the agent
user_intent_caller = await make_agent_caller(user_intent_agent)

In [10]:
# Run the Initial Conversation

session_start = await user_intent_caller.get_session()
print(f"Session Start: {session_start.state}") # expect this to be empty

# We need an async function to await for each conversation
async def run_conversation():
    # start things off by describing your goal
    await user_intent_caller.call("""I'd like a bill of materials graph (BOM graph) which includes all levels from suppliers to finished product, 
    which can support root-cause analysis.""") 

    if PERCEIVED_USER_GOAL not in session_start.state:
        # the LLM may have asked a clarifying question. offer some more details
        await user_intent_caller.call("""I'm concerned about possible manufacturing or supplier issues.""")        

    # Optimistically presume approval.
    await user_intent_caller.call("Approve that goal.", False)

await run_conversation()

session_end = await user_intent_caller.get_session()

Session Start: {}

>>> User Query: I'd like a bill of materials graph (BOM graph) which includes all levels from suppliers to finished product, 
    which can support root-cause analysis.
<<< Agent Response: To better understand your goal, could you provide a bit more information on the specific use cases or challenges you are looking to address with this Bill of Materials (BOM) graph? For instance, are you primarily focused on tracking supply chain disruptions, improving production efficiency, or maybe something else? This will help in framing the graph description accurately.

>>> User Query: I'm concerned about possible manufacturing or supplier issues.
<<< Agent Response: Here is the perceived goal for your Bill of Materials graph:

**Kind of Graph:** Bill of Materials

**Description:** A comprehensive Bill of Materials (BOM) graph that spans all levels from suppliers to finished products. The graph is designed to support root-cause analysis of potential manufacturing or supplier i